In [748]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, ThresholdedReLU, MaxPooling2D, Embedding, Dropout
from keras.optimizers import Adam, SGD, RMSprop
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import gc

In [749]:
df=pd.read_excel("datathon.xlsx",skiprows=3)

In [750]:
df.head(10)

,Cluster,Brand Group,Country,Function,Jan 2012,Feb 2012,Mar 2012,Unnamed: 7,Apr 2012,May 2012,...,Unnamed: 113,Jul 2018,Aug 2018,Sep 2018,Unnamed: 117,Oct 2018,Nov 2018,Dec 2018,Unnamed: 121,Unnamed: 122
0,Cluster 3,Brand Group 12,Country 19,Sales 1,1861.328751,1995.945017,1998.533792,5855.807560,2019.243986,2425.681558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Cluster 3,Brand Group 12,Country 19,Sales 2,1316.247027,1450.223283,1440.837462,4207.307772,1564.066421,2014.132904,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cluster 3,Brand Group 12,Country 19,Investment 1,-216.162658,-238.167239,-217.457045,-671.786942,-214.868270,-390.904926,...,-431.807560,-146.395189,-145.100802,-171.894616,-463.390607,-136.946163,-189.974966,-156.680908,-483.602037,-1816.303068
3,Cluster 3,Brand Group 12,Country 19,Investment 2,-107.434135,-14.238259,-62.130584,-183.802978,-91.901489,-199.335624,...,-155.455899,-35.595647,-64.460481,-111.705613,-211.761741,-29.253150,-133.434491,-88.018327,-250.705968,-696.492909
4,Cluster 3,Brand Group 12,Country 19,Investment 4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,-3.106529,-3.106529,0.000000,0.000000,0.000000,0.000000,-1.423826
5,Cluster 3,Brand Group 12,Country 19,Investment 6,-56.953036,-55.658648,-64.719359,-177.331042,-56.953036,-55.658648,...,-149.890034,-42.197022,-41.032073,-48.539519,-131.768614,-45.044674,-45.007720,-35.730998,-125.783392,-544.129325
6,Cluster 3,Brand Group 17,Country 19,Sales 1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Cluster 3,Brand Group 17,Country 19,Sales 2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Cluster 3,Brand Group 17,Country 19,Investment 1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-6300.688431,-2123.312715,-1912.457045,-2342.970218,-6378.739977,-2114.899198,-2206.086863,-1968.091008,-6289.077068,-24996.207653
9,Cluster 3,Brand Group 17,Country 19,Investment 2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-7063.470791,-1790.266896,-1048.194731,-1904.690722,-4743.152348,-1511.585338,-1595.179450,-1041.791168,-4148.555956,-19825.655613


##  Sales1

In [751]:
df = df[df.columns.drop(list(df.filter(regex='Unnamed')))]
dfsales2=df.loc[df['Function'] == "Sales 2"]
dfsales2 = dfsales2[dfsales2.columns.drop(list(dfsales2.filter(regex='2018')))]
dfsales2=dfsales2.groupby(['Cluster','Brand Group']).agg('sum')
dfmelt=pd.melt(dfsales2.reset_index(), id_vars=["Cluster", "Brand Group"], 
                  var_name="Date", value_name="Value_Y")

In [752]:
dfmelt.head(5)

,Cluster,Brand Group,Date,Value_Y
0,Cluster 1,Brand Group 12,Jan 2012,616.408961
1,Cluster 1,Brand Group 13,Jan 2012,62.381470
2,Cluster 1,Brand Group 15,Jan 2012,0.000000
3,Cluster 1,Brand Group 16,Jan 2012,2457.695992
4,Cluster 1,Brand Group 17,Jan 2012,0.000000


## Sales 2

In [555]:
dfinves=df.loc[df['Function'].str.contains("Investment")]

In [556]:
dfinves=dfinves.groupby(['Cluster','Brand Group','Function']).agg('sum')


In [558]:
dfinves=pd.melt(dfinves.reset_index(), id_vars=["Cluster", "Brand Group","Function"], 
                  var_name="Date", value_name="Investment")

In [567]:
dfinves=dfinves.pivot_table('Investment', ['Cluster', 'Brand Group','Date'], 'Function')

In [681]:
dfinves=dfinves.reset_index()

In [691]:
dfinves=dfinves.fillna(0)

In [699]:
dfinves.head(5)

,Cluster,Brand Group,Date_coded,Investment 1,Investment 2,Investment 3,Investment 4,Investment 5,Investment 6
0,Cluster 1,Brand Group 12,3,-39.664141,-79.814354,0.000000,-1.451180,0.0,-9.745249
1,Cluster 1,Brand Group 12,15,-44.510635,-144.391369,0.000000,-1.462129,0.0,-12.250295
2,Cluster 1,Brand Group 12,27,-51.325172,-118.808299,-0.682549,-3.965413,0.0,-11.078440
3,Cluster 1,Brand Group 12,39,-47.388726,-62.490278,0.000000,-3.320586,0.0,-14.386244
4,Cluster 1,Brand Group 12,51,-55.299492,-115.182981,0.000000,-8.180448,0.0,-7.625617


In [694]:
dfinves=dfinves.replace({"Date": dictmonths})

In [698]:
dfinves.columns=['Cluster', 'Brand Group', 'Date_coded', 'Investment 1', 'Investment 2',
       'Investment 3', 'Investment 4', 'Investment 5', 'Investment 6']

In [ ]:
dfmerged=dfmerged.fillna(0)

In [682]:
dfmerged=pd.merge(dfmelt,dfinves,
                 on=['Cluster','Brand Group','Date'])

In [683]:
dfmerged.head(5)

,Cluster,Brand Group,Date,Value_Y,Investment 1,Investment 2,Investment 3,Investment 4,Investment 5,Investment 6
0,Cluster 1,Brand Group 12,Jan 2012,616.408961,-35.2948,-31.457517,0.0,-0.286406,NaN,-7.976037
1,Cluster 1,Brand Group 15,Jan 2012,0.000000,0.0000,0.000000,NaN,0.000000,NaN,NaN
2,Cluster 1,Brand Group 16,Jan 2012,2457.695992,NaN,0.024731,NaN,-1.545383,NaN,NaN
3,Cluster 1,Brand Group 17,Jan 2012,0.000000,0.0000,0.000000,0.0,0.000000,NaN,0.000000
4,Cluster 1,Brand Group 18,Jan 2012,0.000000,NaN,0.000000,NaN,0.000000,NaN,NaN


In [753]:
dictmonths=dict(zip(dfmelt["Date"].unique(),range(len(dfmelt["Date"].unique()))))

In [754]:
dfmelt["Date_coded"]=dfmelt["Date"].apply(lambda x: dictmonths[x])

In [755]:
dfmelt["Month"]=dfmelt["Date_coded"].apply(lambda x: x%12)

In [756]:
dfmelt

,Cluster,Brand Group,Date,Value_Y,Date_coded,Month
0,Cluster 1,Brand Group 12,Jan 2012,616.408961,0,0
1,Cluster 1,Brand Group 13,Jan 2012,62.381470,0,0
2,Cluster 1,Brand Group 15,Jan 2012,0.000000,0,0
3,Cluster 1,Brand Group 16,Jan 2012,2457.695992,0,0
4,Cluster 1,Brand Group 17,Jan 2012,0.000000,0,0
5,Cluster 1,Brand Group 18,Jan 2012,0.000000,0,0
6,Cluster 1,Brand Group 19,Jan 2012,1328.072408,0,0
7,Cluster 1,Brand Group 24,Jan 2012,0.000000,0,0
8,Cluster 1,Brand Group 25,Jan 2012,1869.355444,0,0
9,Cluster 1,Brand Group 26,Jan 2012,2044.415276,0,0


In [757]:
import numpy as np
dfmelt["Actual"] = np.nan
dfmelt["Lag-1"] = np.nan
dfmelt["Lag-2"] = np.nan
dfmelt["Lag-3"] = np.nan
# dfmelt["Lag-4"] = np.nan

In [758]:
def add_lag_values(dfpre,lag,lagname):
    for i in range (len (dfpre)):
        if dfpre.iloc[i].Date_coded>12:
            dfpre.iloc[i,dfpre.columns.get_loc(lagname)]=dfpre[(dfpre["Cluster"]==dfpre.iloc[i].Cluster) 
                   & (dfpre["Brand Group"]==dfpre.iloc[i,dfpre.columns.get_loc("Brand Group")])
                                                                &(dfpre["Date_coded"]==dfpre.iloc[i].Date_coded-lag)]["Value_Y"].values[0]
    return dfpre

In [759]:
dfpre=add_lag_values(dfmelt,1,'Actual')
dfpre=add_lag_values(dfmelt,2,'Lag-1')
dfpre=add_lag_values(dfpre,3,'Lag-2')
dfpre=add_lag_values(dfpre,4,'Lag-3')

In [761]:
dfpre.to_pickle("lags_pre")

In [762]:
dfpre['Month']=dfpre['Month'].apply(lambda x: 11 if x == 0 else x-1)

In [763]:
dfpre['sales_trend'] = dfpre['Actual']

lag_list=[1,2,3]

for lag in [1,2,3]:
    ft_name = ('Lag-%s' % lag)
    dfpre['sales_trend'] -= dfpre[ft_name]

dfpre['sales_trend'] /= len(lag_list) + 1

In [764]:
dfpre.head(5)

,Cluster,Brand Group,Date,Value_Y,Date_coded,Month,Actual,Lag-1,Lag-2,Lag-3,sales_trend
0,Cluster 1,Brand Group 12,Jan 2012,616.408961,0,10,NaN,NaN,NaN,NaN,NaN
1,Cluster 1,Brand Group 13,Jan 2012,62.381470,0,10,NaN,NaN,NaN,NaN,NaN
2,Cluster 1,Brand Group 15,Jan 2012,0.000000,0,10,NaN,NaN,NaN,NaN,NaN
3,Cluster 1,Brand Group 16,Jan 2012,2457.695992,0,10,NaN,NaN,NaN,NaN,NaN
4,Cluster 1,Brand Group 17,Jan 2012,0.000000,0,10,NaN,NaN,NaN,NaN,NaN


In [767]:
dfpre=dfpre.dropna()

In [768]:
dfpre['Date_coded']=dfpre_sales['Date_coded'].apply(lambda x: x-1)

C:\Users\dsalvadu\AppData\Local\Continuum\anaconda3\envs\CDAC\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [702]:
dfinves.head(5)

,Cluster,Brand Group,Date_coded,Investment 1,Investment 2,Investment 3,Investment 4,Investment 5,Investment 6
0,Cluster 1,Brand Group 12,3,-39.664141,-79.814354,0.000000,-1.451180,0.0,-9.745249
1,Cluster 1,Brand Group 12,15,-44.510635,-144.391369,0.000000,-1.462129,0.0,-12.250295
2,Cluster 1,Brand Group 12,27,-51.325172,-118.808299,-0.682549,-3.965413,0.0,-11.078440
3,Cluster 1,Brand Group 12,39,-47.388726,-62.490278,0.000000,-3.320586,0.0,-14.386244
4,Cluster 1,Brand Group 12,51,-55.299492,-115.182981,0.000000,-8.180448,0.0,-7.625617


In [703]:
dffinal=pd.merge(dfpre_sales,dfinves,
                 on=['Cluster','Brand Group','Date_coded'])

In [704]:
dffinal.head(10)

,Cluster,Brand Group,Date,Value_Y,Date_coded,Month,Actual,Lag-1,Lag-2,Lag-3,sales_trend,Investment 1,Investment 2,Investment 3,Investment 4,Investment 5,Investment 6
0,Cluster 1,Brand Group 12,Feb 2013,886.812584,12,0,745.704077,1084.703889,946.821916,943.346418,-557.292036,-41.558871,-14.368532,0.0,-0.224499,0.0,-9.895950
1,Cluster 1,Brand Group 15,Feb 2013,0.000000,12,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
2,Cluster 1,Brand Group 16,Feb 2013,2297.069386,12,0,1965.631205,2106.791079,2956.718563,2495.493383,-1398.342955,0.000000,0.000000,0.0,-1.298518,0.0,0.000000
3,Cluster 1,Brand Group 17,Feb 2013,0.000000,12,0,0.000000,0.000000,0.000000,0.000000,0.000000,-3.018683,0.000000,0.0,0.000000,0.0,-2.833370
4,Cluster 1,Brand Group 18,Feb 2013,0.000000,12,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000
5,Cluster 1,Brand Group 19,Feb 2013,1498.933024,12,0,1256.505397,1204.286111,1735.955221,1139.976953,-705.928222,-0.876540,0.000000,0.0,-5.600064,0.0,-1.772652
6,Cluster 1,Brand Group 24,Feb 2013,0.000000,12,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-13.167373,0.0,0.000000,0.0,0.000000
7,Cluster 1,Brand Group 25,Feb 2013,2398.767022,12,0,2229.490821,2138.460934,2336.864699,3205.431354,-1362.816541,-32.372728,-0.095763,0.0,-5.871387,0.0,0.000000
8,Cluster 1,Brand Group 26,Feb 2013,2710.350144,12,0,2550.119528,1905.375056,1800.682066,2760.493348,-979.107736,-185.888295,-12.242750,0.0,-3.011869,0.0,-15.900721
9,Cluster 1,Brand Group 27,Feb 2013,977.035873,12,0,1095.882446,517.510205,873.167812,1026.741672,-330.384311,0.000000,-0.009205,0.0,1.102464,0.0,0.000000


In [833]:
dfpre.to_pickle("dataset_1_08.pkl")

In [769]:
yvalues =dfpre["Value_Y"]
xvalues = dfpre.drop(labels=['Value_Y'],axis=1)

In [770]:
from sklearn.preprocessing import StandardScaler
#sc = StandardScaler()
##sc.fit(xvalues[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']])
#xvalues[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']] =sc.transform(xvalues[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']])

In [771]:
X_train, X_valid, y_train, y_valid = train_test_split(xvalues, yvalues, test_size=0.2, shuffle=True)

In [772]:

xvalues.columns

Index(['Cluster', 'Brand Group', 'Date', 'Date_coded', 'Month', 'Actual',
       'Lag-1', 'Lag-2', 'Lag-3', 'sales_trend'],
      dtype='object')

In [ ]:
invest=['Investment 1',
       'Investment 2', 'Investment 3', 'Investment 4', 'Investment 5',
       'Investment 6']

In [773]:
X_train=X_train[['Month','Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']]
X_valid=X_valid[['Month','Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']]

In [774]:
from xgboost import XGBRegressor

In [808]:
model = XGBRegressor(
    max_depth=6,
    n_estimators=1000,
    min_child_weight=20, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.3,    
    seed=42)

model.fit(
    X_train, 
    y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, y_train), (X_valid, y_valid)], 
    verbose=True, 
    early_stopping_rounds = 30)

[0]	validation_0-rmse:3645.8	validation_1-rmse:4296.31
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 30 rounds.
[1]	validation_0-rmse:3299.02	validation_1-rmse:3890.47
[2]	validation_0-rmse:2989.15	validation_1-rmse:3522.78
[3]	validation_0-rmse:2711.28	validation_1-rmse:3191.4
[4]	validation_0-rmse:2460.95	validation_1-rmse:2892.91
[5]	validation_0-rmse:2235.92	validation_1-rmse:2627.01
[6]	validation_0-rmse:2033.71	validation_1-rmse:2387.21
[7]	validation_0-rmse:1854.34	validation_1-rmse:2174.73
[8]	validation_0-rmse:1695.63	validation_1-rmse:1984.69
[9]	validation_0-rmse:1553.43	validation_1-rmse:1812.46
[10]	validation_0-rmse:1427.85	validation_1-rmse:1664.76
[11]	validation_0-rmse:1314.65	validation_1-rmse:1528.61
[12]	validation_0-rmse:1216.35	validation_1-rmse:1411.78
[13]	validation_0-rmse:1129.56	validation_1-rmse:1310.97
[14]	validation_0-rmse:1052.56	validation_1-rmse:1218.5

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eta=0.3, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=6, min_child_weight=20, missing=None,
       n_estimators=1000, n_jobs=1, nthread=None, objective='reg:linear',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=42, silent=True, subsample=0.8)

# Pred dataset

In [809]:
dfsales2=dfsales2.groupby(['Cluster','Brand Group']).agg('sum')

In [810]:
dfsales2

Jan 2012      Feb 2012      Mar 2012  \
Cluster   Brand Group                                                
Cluster 1 Brand Group 12    616.408961    685.081035    968.938870   
          Brand Group 13     62.381470     69.445132     42.081080   
          Brand Group 15      0.000000      0.000000      0.000000   
          Brand Group 16   2457.695992   2616.171336   2019.736343   
          Brand Group 17      0.000000      0.000000      0.000000   
          Brand Group 18      0.000000      0.000000      0.000000   
          Brand Group 19   1328.072408   1747.087079   1843.272270   
          Brand Group 24      0.000000      0.000000      0.000000   
          Brand Group 25   1869.355444   1724.577575   3067.466622   
          Brand Group 26   2044.415276   2124.060822   2773.263487   
          Brand Group 27    616.696052    943.158430    991.609400   
          Brand Group 28      0.000000      0.000000      0.000000   
          Brand Group 30   1451.344742   2421.703682   2174.680037   
          Brand Group 31    601.508882   1221.516101    330.240589   
          Brand Group 36    224.091013     10.742677      3.547430   
          Brand Group 38    158.842381    140.429788    136.151464   
          Brand Group 39    667.435844    696.507436    618.169416   
          Brand Group 40    203.767645    186.701557    136.532992   
          Brand Group 41   2427.396563   3221.070147   3093.632493   
          Brand Group 43      0.000000      0.000000      0.000000   
          Brand Group 44    576.048693    871.090969   1361.650537   
          Brand Group 45   1995.503104   1663.363557   1307.480906   
          Brand Group 46   1335.533122   1394.229661   1505.275423   
          Brand Group 51    368.564296    400.115372    594.180684   
          Brand Group 58   1997.748581   1305.279925   1295.629396   
          Brand Group 63  -2408.708191    -69.668974    521.596040   
          Brand Group 70    553.832361   1916.631439    991.594098   
          Brand Group 71     40.007880     36.881432     74.881322   
          Brand Group 72    136.189910     68.737912    131.765769   
          Brand Group 73      0.000000      0.000000      0.000000   
...                                ...           ...           ...   
Cluster 9 Brand Group 17      0.000000      0.000000      0.000000   
          Brand Group 19   1065.970694    965.448115    826.204342   
          Brand Group 24      0.000000      0.000000      0.000000   
          Brand Group 25   1311.951469   1228.487730   1596.800828   
          Brand Group 26    578.979771    543.785442    501.128984   
          Brand Group 27    247.781298    204.107500    311.394846   
          Brand Group 30    849.946315    827.654128    931.822650   
          Brand Group 31     33.307956     37.928092     34.382406   
          Brand Group 36    411.032237    203.143857    326.407941   
          Brand Group 38     99.350845     45.914903     62.179995   
          Brand Group 39     49.729809     58.723182     58.563123   
          Brand Group 41  21390.272688  23210.413172  26803.499781   
          Brand Group 45    530.340734    465.930587    457.566091   
          Brand Group 46   2933.358538   2485.503539   3695.535976   
          Brand Group 51    109.127156    144.024336    126.068284   
          Brand Group 58    600.962268    495.003715    606.023913   
          Brand Group 70    123.079074    102.217856    118.692469   
          Brand Group 71    115.778040    101.534790    104.466731   
          Brand Group 72      1.273245      2.122076      1.697661   
          Brand Group 73      0.000000      0.000000      0.000000   
          Brand Group 76    395.554874    233.415453    362.856133   
          Brand Group 78     59.237114     50.176649     73.802040   
          Brand Group 79      1.564914      0.959141      1.682703   
          Brand Group 84   1223.608330   1416.356354   1882.021613   
          Brand Group 87   1350.142011  

In [811]:
dfpred=pd.melt(dfsales2.reset_index(), id_vars=["Cluster", "Brand Group"], 
                  var_name="Date", value_name="Value")

In [812]:
dfsales2.reset_index()

,Cluster,Brand Group,Jan 2012,Feb 2012,Mar 2012,Apr 2012,May 2012,Jun 2012,Jul 2012,Aug 2012,...,Mar 2017,Apr 2017,May 2017,Jun 2017,Jul 2017,Aug 2017,Sep 2017,Oct 2017,Nov 2017,Dec 2017
0,Cluster 1,Brand Group 12,616.408961,685.081035,968.938870,723.701936,656.607506,319.401731,953.191022,658.304347,...,1494.614561,1583.525099,1396.117515,1471.551885,1632.173591,1627.574308,1608.715411,1538.061964,1574.399619,1.381174e+03
1,Cluster 1,Brand Group 13,62.381470,69.445132,42.081080,55.139918,65.677360,36.194606,46.670999,62.483972,...,5.496543,14.480741,9.765432,11.746420,7.310123,10.965185,5.803457,5.608148,8.537778,6.361481e+00
2,Cluster 1,Brand Group 15,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,613.388915,-229.544362,798.197908,-193.611296,180.088396,-126.788616,35.634313,177.382970,48.785139,-2.653626e+02
3,Cluster 1,Brand Group 16,2457.695992,2616.171336,2019.736343,2521.239601,2341.879305,1909.803910,2335.718836,2284.694096,...,556.856275,516.432602,711.261828,575.720936,638.983822,590.373449,612.722539,535.401404,566.573007,5.391845e+02
4,Cluster 1,Brand Group 17,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1632.612631,1990.658124,1586.519705,1959.033915,2334.987199,2371.470463,2283.761079,2830.252855,2530.420686,2.870711e+03
5,Cluster 1,Brand Group 18,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,8.704348,-5.247172,0.447933,3.171981,-68.883897,40.824615,-0.139374,0.011042,-23.587434,3.298240e+02
6,Cluster 1,Brand Group 19,1328.072408,1747.087079,1843.272270,2039.346028,1691.095349,2047.555687,1610.354841,1770.380294,...,893.457850,855.506720,802.064436,1066.648259,785.950844,886.596334,755.373756,959.422827,953.348555,6.881483e+02
7,Cluster 1,Brand Group 24,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,535.500112,617.733001,744.317609,906.058486,837.815300,754.027434,1193.776122,1224.923058,1434.480782,1.314431e+03
8,Cluster 1,Brand Group 25,1869.355444,1724.577575,3067.466622,2478.010511,1714.321134,1882.068631,2453.273551,2329.189122,...,1425.465837,1150.809671,1356.505256,1254.493173,1282.958362,1269.700506,1429.508574,1275.457380,1358.765291,1.249301e+03
9,Cluster 1,Brand Group 26,2044.415276,2124.060822,2773.263487,2272.917754,2528.122320,2218.761723,2335.513978,2267.608240,...,2842.389283,2684.698994,2723.677431,2740.300859,2542.729046,2499.681858,2571.601476,2224.735039,2913.654991,2.441371e+03


In [813]:
dfindexes=dfpred=dfsales2.reset_index()[["Cluster","Brand Group"]]

In [814]:
# GET LABELSSSSSSSSSSSSSSSs

In [815]:
dfpred=dfsales2.reset_index()[["Sep 2017","Oct 2017","Nov 2017","Dec 2017"]]

In [816]:
dfpred["Month"]=11

In [817]:
dfpred

,Sep 2017,Oct 2017,Nov 2017,Dec 2017,Month
0,1608.715411,1538.061964,1574.399619,1.381174e+03,11
1,5.803457,5.608148,8.537778,6.361481e+00,11
2,35.634313,177.382970,48.785139,-2.653626e+02,11
3,612.722539,535.401404,566.573007,5.391845e+02,11
4,2283.761079,2830.252855,2530.420686,2.870711e+03,11
5,-0.139374,0.011042,-23.587434,3.298240e+02,11
6,755.373756,959.422827,953.348555,6.881483e+02,11
7,1193.776122,1224.923058,1434.480782,1.314431e+03,11
8,1429.508574,1275.457380,1358.765291,1.249301e+03,11
9,2571.601476,2224.735039,2913.654991,2.441371e+03,11


In [818]:
dfpred.columns=["Lag-3","Lag-2","Lag-1","Actual","Month"]

In [819]:
dfpred['sales_trend'] = dfpred['Actual']

lag_list=[1,2,3]

for lag in [1,2,3]:
    ft_name = ('Lag-%s' % lag)
    dfpre['sales_trend'] -= dfpre[ft_name]

dfpre['sales_trend'] /= len(lag_list) + 1

C:\Users\dsalvadu\AppData\Local\Continuum\anaconda3\envs\CDAC\lib\site-packages\ipykernel\__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\dsalvadu\AppData\Local\Continuum\anaconda3\envs\CDAC\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [820]:
dfpred.head(5)

,Lag-3,Lag-2,Lag-1,Actual,Month,sales_trend
0,1608.715411,1538.061964,1574.399619,1381.174249,11,1381.174249
1,5.803457,5.608148,8.537778,6.361481,11,6.361481
2,35.634313,177.382970,48.785139,-265.362597,11,-265.362597
3,612.722539,535.401404,566.573007,539.184516,11,539.184516
4,2283.761079,2830.252855,2530.420686,2870.710849,11,2870.710849


In [821]:
#dfpred[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']] =sc.transform(dfpred[['Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']])

In [822]:
def add_values(pred_x, new_pred,month):
    pred_x['Month']=month
    
    pred_x["Lag-3"]=pred_x["Lag-2"]
    pred_x["Lag-2"]=pred_x["Lag-1"]
    pred_x["Lag-1"]=pred_x["Actual"]
    pred_x["Actual"]=new_pred

    pred_x['sales_trend'] = pred_x['Actual']
    lag_list=[1,2,3]

    for lag in [1,2,3]:
        ft_name = ('Lag-%s' % lag)
        pred_x['sales_trend'] -= pred_x[ft_name]

    pred_x['sales_trend'] /= len(lag_list) + 1
    
    return pred_x

In [823]:
list_predictions=[]

In [824]:
xpred=dfpred[['Month', 'Lag-1', 'Lag-2', 'Lag-3', 'Actual', 'sales_trend']]

In [825]:
for i in range(12):
    preds=model.predict(xpred)
    list_predictions.append(preds)
    xpred=add_values(xpred, preds,i)

In [826]:
len(list_predictions[0])

303

In [827]:
for d in range(len(list_predictions)):
    dfindexes['Month-%s' % d]=list_predictions[d]

In [828]:
dfindexes["Brand Group"].unique()

array(['Brand Group 12', 'Brand Group 13', 'Brand Group 15',
       'Brand Group 16', 'Brand Group 17', 'Brand Group 18',
       'Brand Group 19', 'Brand Group 24', 'Brand Group 25',
       'Brand Group 26', 'Brand Group 27', 'Brand Group 28',
       'Brand Group 30', 'Brand Group 31', 'Brand Group 36',
       'Brand Group 38', 'Brand Group 39', 'Brand Group 40',
       'Brand Group 41', 'Brand Group 43', 'Brand Group 44',
       'Brand Group 45', 'Brand Group 46', 'Brand Group 51',
       'Brand Group 58', 'Brand Group 63', 'Brand Group 70',
       'Brand Group 71', 'Brand Group 72', 'Brand Group 73',
       'Brand Group 76', 'Brand Group 78', 'Brand Group 79',
       'Brand Group 80', 'Brand Group 84', 'Brand Group 85',
       'Brand Group 87', 'Brand Group 89', 'Brand Group 90',
       'Brand Group 92', 'Brand Group 94', 'Brand Group 96',
       'Brand Group 97', 'Brand Group 20', 'Brand Group 98',
       'Brand Group 60', 'Brand Group 64', 'Brand Group 21'], dtype=object)

In [829]:
dfindexes

,Cluster,Brand Group,Month-0,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11
0,Cluster 1,Brand Group 12,1829.865479,1522.889526,1623.515503,1652.489014,1591.738525,1603.259277,1602.356689,1623.089722,1623.089722,1623.089722,1609.321289,1530.416748
1,Cluster 1,Brand Group 13,33.549194,33.900761,42.992546,52.014080,64.340218,80.160789,80.160789,99.646996,123.956551,123.956551,136.961472,123.509262
2,Cluster 1,Brand Group 15,273.112701,228.817307,205.897263,180.685394,222.225143,210.408310,206.379654,210.408310,228.469742,210.408310,228.469742,196.956100
3,Cluster 1,Brand Group 16,798.643799,632.209534,700.258423,698.297302,640.537048,640.537048,640.537048,642.560059,642.560059,642.560059,642.560059,619.412598
4,Cluster 1,Brand Group 17,2604.358887,2708.258057,2764.504883,2812.302246,2831.850830,2933.670410,2636.034424,2774.421143,2818.750000,2792.366455,2837.480957,2835.931641
5,Cluster 1,Brand Group 18,488.820984,403.666809,519.228882,461.014587,419.846680,451.127258,452.943756,424.950317,430.364471,425.356537,419.942383,411.904297
6,Cluster 1,Brand Group 19,1468.766235,964.605835,1040.522461,1172.920410,1163.836426,1106.307251,1115.974365,1138.576904,1138.576904,1138.576904,1138.576904,1100.486572
7,Cluster 1,Brand Group 24,1730.091675,1415.171509,1550.319092,1564.252197,1539.179321,1584.101440,1505.134521,1549.904541,1532.809448,1545.910278,1532.255981,1469.012329
8,Cluster 1,Brand Group 25,1698.744385,1396.702881,1528.695923,1549.651123,1480.611450,1540.365479,1507.928589,1487.342285,1478.121582,1474.127319,1474.127319,1392.577026
9,Cluster 1,Brand Group 26,2555.331787,2576.621094,2626.912842,2541.986572,2632.965576,2621.613037,2375.978027,2552.397705,2457.031738,2463.106689,2508.257324,2428.780518


In [830]:
dfindexes.iloc[0][2:]

Month-0     1829.87
Month-1     1522.89
Month-2     1623.52
Month-3     1652.49
Month-4     1591.74
Month-5     1603.26
Month-6     1602.36
Month-7     1623.09
Month-8     1623.09
Month-9     1623.09
Month-10    1609.32
Month-11    1530.42
Name: 0, dtype: object

In [831]:
dir_submission_template = 'Data_Novartis_Datathon-Results_Challenge1_Template.csv'

submission_template = pd.read_csv(dir_submission_template)

bg_set = set()

#a = pd.DataFrame(final_res, columns=['Cluster', 'Brand Group', 'res'])

for i, r in submission_template.iterrows():
    bg = r['Brand Group'][12:].replace(',', '').split(' ')
    bg_set.update(bg)
    bg = [ 'Brand Group ' + str(i) for i in bg ]
    print(r['Cluster'], r['Brand Group'], bg)
    
#    print(type(r['res']))
    
    res = dfindexes[dfindexes['Cluster'] == r['Cluster']]
    res = [ rs[2:] for j, rs in res.iterrows() if rs['Brand Group'] in bg ]
#    res = res['res']
    res = np.sum(res, axis=0)
    submission_template.iloc[i, 2:] = res
    
    
for i, r in submission_template[submission_template['Brand Group'] == 'others'].iterrows():
    print(r.Cluster)
    
    res = dfindexes[dfindexes['Cluster'] == r['Cluster']]
    res1 = [ rs[2:] for j, rs in res.iterrows() if rs['Brand Group'][12:] not in bg_set ]

    res2 = np.sum(res1, axis=0)
    submission_template.iloc[i, 2:] = res2

Cluster 1 Brand Group 17 ['Brand Group 17']
Cluster 2 Brand Group 17 ['Brand Group 17']
Cluster 3 Brand Group 17 ['Brand Group 17']
Cluster 4 Brand Group 17 ['Brand Group 17']
Cluster 5 Brand Group 17 ['Brand Group 17']
Cluster 8 Brand Group 17 ['Brand Group 17']
Cluster 9 Brand Group 17 ['Brand Group 17']
Cluster 10 Brand Group 17 ['Brand Group 17']
Cluster 1 Brand Group 24 ['Brand Group 24']
Cluster 3 Brand Group 24 ['Brand Group 24']
Cluster 4 Brand Group 24 ['Brand Group 24']
Cluster 5 Brand Group 24 ['Brand Group 24']
Cluster 8 Brand Group 24 ['Brand Group 24']
Cluster 9 Brand Group 24 ['Brand Group 24']
Cluster 10 Brand Group 24 ['Brand Group 24']
Cluster 1 Brand Group 30 ['Brand Group 30']
Cluster 2 Brand Group 30 ['Brand Group 30']
Cluster 4 Brand Group 30 ['Brand Group 30']
Cluster 5 Brand Group 30 ['Brand Group 30']
Cluster 7 Brand Group 30 ['Brand Group 30']
Cluster 8 Brand Group 30 ['Brand Group 30']
Cluster 9 Brand Group 30 ['Brand Group 30']
Cluster 10 Brand Group 30 ['Br

In [832]:
submission_template.to_csv("submission_3.csv",index=False)